# Generic workflow

Key Features of this workflow:
* The Generic workflow is pathogen and model agnostic. As such, the outputs report will consist of only histograms (with KDEs) and displays of median with 0.95 HDI.
* Minimal requirements: a single ready to launch XML
* This workflow cannot segregate data into multiple XMLs to be run in parallel, see `Generic-Comparative.ipynb` for those options.
* All phases available (initial tree building, downsampling, XML alterations, configure no. of runs, etc.)
* Any changes to modelling priors are to be done first in Beauti (future feature), MCMC and other options can be changed below.

<details>
    <summary>Click To See A Decription of Parameters</summary>
        <pre>
            <code>

Running an Instance of this Workflow
-------------------------------------------
overall_save_dir: str
    Path to where you are saving all the runs of this workflow.
    This creates a folder by the name you specify here e.g. creates a folder named folder in the root folder to save all the files produced when running the workflow.

specific_run_save_dir: str, optional
    Sub-directory of overall_save_dir you wish to save all the files from this instance of this workflow.
    If None, 'None' or an empty string a timestamp of format 'YYYY-MM-DD_hour-min-sec' is used instead.

max_threads: int, default None
    The maximum number of threads to use when calling gnu parallel in phases 2i and 4. If not given and BEAST_pype is running
    in a SLURM job the SLURM environment variable `SLURM_CPUS_PER_TASK` is used. If not given and BEAST_pype is NOT running in
    a SLURM job the number of cores available minus 1 is used (`multiprocessing.cpu_count() - 1`).

kernel_name: str, default 'beast_pype'
    Name of Jupyter python kernel to use when running workflow. This is also the name of the conda environment to use in phases 4 &
    phase 2ii (as these Jupyter notebooks use the `bash` kernel).

General Inputs
----------------
ready_to_go_xml: str, optional
    Path to a BEAST 2 xml that you wish to run unaltered. If provided phases 2i, 2ii and 3 are skipped.

template_xml_path: str, (required if not using a `ready_to_go_xml`)
    Path to template BEAST 2 xml.

fasta_path: str, (required if not using a `ready_to_go_xml`)
    Path to fasta file containing sequences to be placed into template xml.

metadata_path: str, (required if not using a `ready_to_go_xml`)
      Path to csv or tsv containing metadata for sequences in fasta_path.

sample_id_field: str, (required if not using a `ready_to_go_xml`)
    Name of field in metadata_db containing sequence IDs.

collection_date_field: str, (required if not using a `ready_to_go_xml`)
    Name of field in metadata_db containing collection dates of sequences. Should be formatted YYYY-MM-DD.

Add pre-made Initial Tree
--------------------

initial_tree_path: str, optional
    Path to initial tree to use in generating a BEAST 2 xml. Should be .nwk file (Newick format).
    If provided phases 2i and 2ii are skipped.
    If a distance tree is used set initial_tree_type to 'Distance'.
    If a temporal tree is used set initial_tree_type to 'Temporal'.

Initial Tree Building & Downsampling
------------------
use_initial_tree:  bool, default True
    If False an initial tree will not be generated skipping Phases 2i and 2ii. As such, in phase 4 BEAST 2 generate its own
    initial tree.

initial_tree_type: str (either 'Distance' or 'Temporal') or None, default 'Temporal'
    Intial tree type to use.
    If 'Distance' and initial_tree_path is not provided the IQtree tree from Phase-2i-IQTree.ipynb is used for the
    initial tree and phase 2ii is skipped.
    if 'Temporal' and initial_tree_path is not provided the TreeTime tree from Phase-2ii-TreeTime-Tree-Building.ipynb
    is used for the initial tree.

initial_tree_path: str, opional
    Path to initial tree to use in generating a BEAST 2 xml. Should be .nwk file (Newick format).
    If provided phases 2i and 2ii are skiped.
    If a distance tree is used set initial_tree_type to 'Distance'.
    If a temporal tree is used set initial_tree_type to 'Temporal'.

root_strain_names: list of strings, optional
     IDs of sequences used to root 'Temporal' initial_tree. These sequences/nodes are removed from fasta file and initial tree file used to generate
    the BEAST 2 xml.

down_sample_to: int, optional
    Available if not using a `ready_to_go_xml` and generating an initial tree file.
    If not given downsampling will not occur. When given down sampling only occurs if there are more sequences than the value given.
    If downsampling occurs the following are saved in  '{overall_save_dir}/{specific_run_save_dir}/' and used in generating
    a BEAST 2 xml in phase 3:
        initial_trees/down_sampled_time.nwk:  A downsampled temporal tree.
        down_sampled_sequences.fasta:  Fasta file containing downsamplec sequences.
        down_sampled_metadata.csv:  the down sampled metadata.


MCMC Tree/Logfile names Chain-lengths & Save Steps
------------------
log_file_basename: # str, (required if not using a `ready_to_go_xml`)
    If provided .tree, .log and .state files from running BEAST 2 will have this name prefixed by 'run-with-seed-{seed}-', number being that of the chain.
    Must not contain whitespace.

chain_length: # int, (optional, not used if using a `ready_to_go_xml`)
    Number of chains to use for BEAST runs (e.g. 50000000).
    If not given value in template_xml_path will be used.

trace_log_every: # int, (optional, not used if using a `ready_to_go_xml`)
    How often to save a log file during BEAST runs (e.g. 5000).
    If not given value in template_xml_path will be used.

tree_log_every: # int, (optional, not used if using a `ready_to_go_xml`)
    How often to save a tree file during BEAST runs (e.g. 5000).
    If not given value in template_xml_path will be used.

screen_log_every: # int, (optional, not used if using a `ready_to_go_xml`)
    How often to output to screen during BEAST runs (e.g. 5000).
    If not given value in template_xml_path will be used.

store_state_every: # int, (optional, not used if using a `ready_to_go_xml`)
    How often to store MCMC state during BEAST runs (e.g. 5000).
    If not given value in template_xml_path will be used.

Running BEAST 2
--------------------
number_of_beast_runs: int
    Number of chains to use (number of parallel runs to do) when running BEAST (e.g. 9).

seeds: list of ints
    Seeds to use when running BEAST.
    If given, length of list should be the same as the number of chains (so each run has a designated seed).

beast_options_without_a_value: list of strs
    Options not requiring a value to pass to BEAST 2.
     For instance to use a GPU when running BEAST 2 this would be `['-beagle_GPU']`.
    See https://www.beast2.org/2021/03/31/command-line-options.html.

beast_options_needing_a_value: dict
    Options requiring a value to pass to BEAST 2.
    For instance to use 3 threads when running BEAST 2 this would be: `{'-threads': 3}`.
    See https://www.beast2.org/2021/03/31/command-line-options.html.

sbatch_options_without_a_value: list of strs
   Options not requiring a value to pass to sbatch.
    See https://slurm.schedmd.com/sbatch.html.

sbatch_options_needing_a_value: dict
    Options requiring a value to pass to sbatch.
    See https://slurm.schedmd.com/sbatch.html.

  </code>
</pre>

In [ ]:
'''
Parameters
-------------
'''
# Running an Instance of this Workflow
overall_save_dir = None
specific_run_save_dir=None
max_threads=None
kernel_name = 'beast_pype'

# General Inputs
ready_to_go_xml = None
template_xml_path = None
fasta_path= None
metadata_path = None
sample_id_field='strain'
collection_date_field='date'
initial_tree_path = None

# Initial Tree Building & Downsampling
use_initial_tree = True
initial_tree_type = 'Temporal'
root_strain_names = None
remove_root = False
down_sample_to=None

# MCMC Tree/Logfile names Chain-lengths & Save Steps
log_file_basename=None
chain_length = None
trace_log_every = None
tree_log_every = None
screen_log_every = None
store_state_every = None

# Running BEAST 2
number_of_beast_runs = None
seeds = None
beast_options_without_a_value=None
beast_options_needing_a_value=None
sbatch_options_without_a_value=None
sbatch_options_needing_a_value=None

# Choosing a specific report template
report_template = None

# Setup
## Creat Dictionary of Parameters

This needs to be done before importing packages

In [ ]:
parameters = %who_ls
parameters = {var: eval(var) for var in parameters}

## Import Packages

In [ ]:
from beast_pype.nb_utils import execute_notebook
from time import perf_counter
import pandas as pd
import shutil
import importlib.resources as importlib_resources
from beast_pype.workflow_params import GenericWorkflowParams
from beast_pype.diagnostics import gen_beast_diagnostic_nb

## Check, Setup and Record parameters

In [ ]:
parameters = GenericWorkflowParams(**parameters)

### Creating a record for runtimes

This record list of dictionaries will be turned into a pandas dataframe and saved as a csv at the end of this notebook.

In [ ]:
runtime_records = []

### Set path to workflow modules

In [ ]:
workflow_modules = importlib_resources.path('beast_pype', 'workflow_modules')

## Phase 2: Data Pre-Processing
### Phase 2i: Building an IQ Tree tree.
#### Placing Phase 2i Parameters in a Dictionary

In [ ]:
if parameters.ready_to_go_xml is None and \
    parameters.use_initial_tree and \
        (parameters.initial_tree_path is None and parameters.initial_tree_type is not None):
    phase_2i_start = perf_counter()
    phase_2i_params = parameters.retrieve_phase_2i_params()

#### Running Phase 2i.

In [ ]:
#papermill_description=Phase-2i-IQTree-Building
if parameters.ready_to_go_xml is None and \
        parameters.use_initial_tree and \
        (parameters.initial_tree_path is None and parameters.initial_tree_type is not None):
    phase_2i_IQTree_Building_log = execute_notebook(input_path=f'{workflow_modules}/Phase-2i-IQTree-Building.ipynb',
                                                       output_path=parameters.save_dir + '/Phase-2i-IQTree-Building.ipynb',
                                                       parameters=phase_2i_params,
                                                       progress_bar=True,
                                                       nest_asyncio=True
                                                       )
    phase_2i_IQTree_Correction_log = execute_notebook(input_path=f'{workflow_modules}/Phase-2i-IQTree-Correction.ipynb',
                                                         output_path=parameters.save_dir + '/Phase-2i-IQTree-Correction.ipynb',
                                                         parameters={
                                                             'fasta_path': parameters.fasta_path,
                                                             'tree_path': f'{parameters.save_dir}/initial_trees/iqtree.nwk'
                                                         },
                                                         progress_bar=True,
                                                         nest_asyncio=True,
                                                         kernel_name=kernel_name
                                                         )

### Record runtime

In [ ]:
if parameters.ready_to_go_xml is None and \
    parameters.use_initial_tree and \
        (parameters.initial_tree_path is None and parameters.initial_tree_type is not None):

    runtime_records.append({
        'Phase': 'Phase-2i-IQTree-Building.ipynb',
        'Sample': None,
        'Chain': None,
        'Runtime': perf_counter() - phase_2i_start
    })

### Phase 2ii: Building an TreeTime tree and Downsampling.
#### Placing Phase 2ii Parameters in a Dictionary

In [ ]:
if parameters.ready_to_go_xml is None and \
        parameters.use_initial_tree and \
        (parameters.initial_tree_path is None and parameters.initial_tree_type =='Temporal'):
    phase_2ii_start = perf_counter()
    phase_2ii_params = parameters.retrieve_phase_2ii_params()

#### Running Phase 2ii.

In [ ]:
#papermill_description=Phase-2ii-TreeTime-and-Down-Sampling
if parameters.ready_to_go_xml is None and \
        parameters.use_initial_tree and \
        (parameters.initial_tree_path is None and parameters.initial_tree_type =='Temporal'):
    phase_2ii_log = execute_notebook(input_path=f'{workflow_modules}/Phase-2ii-TreeTime-and-Down-Sampling.ipynb',
                                        output_path=parameters.save_dir + '/Phase-2ii-TreeTime-and-Down-Sampling.ipynb',
                                        parameters=phase_2ii_params,
                                        progress_bar=True,
                                        nest_asyncio=True,
                                        kernel_name=kernel_name
                                        )

    runtime_records.append({
        'Phase': 'Phase-2ii-TreeTime-and-Down-Sampling.ipynb',
        'Sample': None,
        'Chain': None,
        'Runtime': perf_counter() - phase_2i_start
    })

## Phase 3 Generating BEAST xmls

### Placing Phase 3 Parameters in a Dictionary

In [ ]:
if parameters.ready_to_go_xml is None:
    phase_3_start = perf_counter()

    phase_3_params = parameters.retrieve_phase_3_params()

    if parameters.down_sample_to is None:
        phase_3_params['fasta_path'] = fasta_path
        phase_3_params['metadata_path'] = metadata_path
    else:
        phase_3_params['fasta_path'] = f'{parameters.save_dir}/down_sampled_sequences.fasta'
        phase_3_params['metadata_path'] =  f'{parameters.save_dir}/down_sampled_metadata.csv'

### Running Phase 3.

In [ ]:
#papermill_description=Phase-3-Generating-XML
if parameters.ready_to_go_xml is None:
    phase_3_log = execute_notebook(input_path=f'{workflow_modules}/Phase-3-Gen-Generic-xml.ipynb',
                                   output_path=parameters.save_dir + '/Phase-3-Gen-Generic-xml.ipynb',
                                   parameters=phase_3_params,
                                   progress_bar=True,
                                   nest_asyncio=True,
                                   kernel_name=kernel_name)
    runtime_records.append({
        'Phase': 'Phase-3-Gen-Generic-xml.ipynb',
        'Sample': None,
        'Chain': None,
        'Runtime': perf_counter() - phase_3_start
    })

## Phase 4 Running BEAST

#### If parameters.ready_to_go_xml was provided save a copy for use as f'{parameters.save_dir}/beast.xml'.

In [ ]:
phase_4_start = perf_counter()
if parameters.ready_to_go_xml is not None:
    shutil.copy(parameters.ready_to_go_xml, f'{parameters.save_dir}/beast.xml')

### Placing Phase 4 Parameters in a Dictionary

In [ ]:
phase_4_params = parameters.retrieve_phase_4_params()

### Running Phase 4.

In [ ]:
#papermill_description=Phase-4-Running-BEAST
if 'sbatch_arg_string' in phase_4_params:
    phase_4_log = execute_notebook(input_path=f'{workflow_modules }/Phase-4-SBATCH-Running-BEAST.ipynb',
                                      output_path=parameters.save_dir + '/Phase-4-SBATCH-Running-BEAST.ipynb',
                                      parameters=phase_4_params,
                                      progress_bar=True,
                                      nest_asyncio=True)
else:
    phase_4_log = execute_notebook(input_path=f'{workflow_modules }/Phase-4-GNU-Parallel-Running-BEAST.ipynb',
                                      output_path=parameters.save_dir + '/Phase-4-GNU-Parallel-Running-BEAST.ipynb',
                                      parameters=phase_4_params,
                                      progress_bar=True,
                                      nest_asyncio=True)
runtime_records.append({
        'Phase': 'Phase-4-Running-BEAST',
        'Sample': None,
        'Chain': None,
        'Runtime': perf_counter() - phase_4_start
    })

## Phase 5: Diagnosing Outputs and Generate Report

Currently, this has to be performed manually. That being said, the code cell below will parameterize a copy of the notebook ready to run. See below for location.

In [ ]:
phase_5_params = parameters.retrieve_phase_5_params()
gen_beast_diagnostic_nb(parameters.save_dir, **phase_5_params)
print(f'Phase 5 notebook is ready for manual use at: \n{parameters.save_dir}/Phase-5-Diagnosing-Outputs-and-Generate-Report.ipynb')

## Recording Runtimes

Converting to pandas DataFrame and saving as CSV.

In [ ]:
runtime_df = pd.DataFrame.from_records(runtime_records)
runtime_df.to_csv(parameters.save_dir + "/runtimes.csv", index=False)